In [1]:
import argparse
from os import device_encoding
import numpy as np
from data_loader import load_data
from train import train
import torch

from parser import parse_args
import random

seed = 2020
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

/home/kimheesu/anaconda3/envs/tf2.5/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
args = argparse.Namespace

args.dataset = "naver-toy"
args.dim=16
args.n_hop=2
args.kge_weight=0.01
args.l2_weight=1e-7
args.lr=0.0001
args.batch_size=256
args.n_epoch=10
args.n_memory=32
args.item_update_mode="plus_transform"
args.using_all_hops=True
args.show_topk=True